In [4]:
import pickle

#Charging needed files

with open("emission.bin", "rb") as fichier:
    emit_p = pickle.load(fichier)

with open("transmition.bin", "rb") as fichier:
    trans_p = pickle.load(fichier)

with open("starter.bin", "rb") as fichier:
    start_p = pickle.load(fichier)

with open("indice_state.bin", "rb") as fichier:
    states = pickle.load(fichier)

with open("indice_word.bin", "rb") as fichier:
    words = pickle.load(fichier)

#Inversion of the word array
i = 0
InvWords= {}
for word in words:
       InvWords[i] = word
       i += 1 

In [5]:
#This function returns the first word of obs and deletes it from obs.
def is_Genuine_Word_V2(obs,p, emit_p):
    beforeToken = ""
    beforeIndice = -1
    
    for i in range(len(emit_p[p])):
        if(float(emit_p[p][i]) > 0):
            if len(obs) >= len(InvWords[i]) and InvWords[i] == obs[0:len(InvWords[i])]:
                if len(InvWords[i]) > len(beforeToken):
                    beforeToken = InvWords[i]
                    beforeIndice = i;
    if len(obs) == len(beforeToken):
        return [beforeToken, beforeIndice, '']
    else:
        return [beforeToken, beforeIndice, obs[-(len(obs)-len(beforeToken)):]]

In [6]:
def viterbi(obs, states = states, start_p = start_p, trans_p = trans_p, emit_p = emit_p):
    wordsLeft = False
    V = [{}]
    path = {}
    token = {}
    state = ""
    prevState = ""
    tmp = obs

    try:
        for s , p in states.items():
                bundle = is_Genuine_Word_V2(obs,p, emit_p)
                V[0][s] = float(start_p[s]) * float(emit_p[p][bundle[1]])  
                path[s] = [s]
                token[s] = [bundle[0]]
                if len(bundle[2]) < len(tmp):
                    tmp = bundle
    
        if tmp == obs:
            raise ValueError("Any token has been found in this input string: "+obs)

        t = 1
        obs = tmp[2]
        bundle = tmp

        while bundle[2] != '':
            V.append({})
            newpath = {}
            newtoken= {}
            newbundle = []
            m = 0
            for s , p in states.items():
                bundle = is_Genuine_Word_V2(obs,p, emit_p)
                if bundle[0] != '':
                    if m < len(bundle[0]):
                        m = len(bundle[0])
                        newbundle = bundle

            if newbundle == []:
                raise ValueError("Any token has been found in this part of input string: "+obs)

            for s , p in states.items():
                (prob, state) = max([(V[t-1][s0] * float(trans_p[p0][p]) * float(emit_p[p][newbundle[1]]), s0) for s0 , p0 in states.items()])
                V[t][s] = prob
                newpath[s] = path[state] + [s]
                bundle = newbundle
                newtoken[s] = token[state] + [bundle[0]]
                
            path = newpath
            token = newtoken
            obs = bundle[2]
            
            t += 1
        
        (prob, state) = max([(V[len(V) - 1][s], s) for s, p in states.items()])
        return (token[state], path[state])
            
    except ValueError as e:
        print(e)  

In [7]:
viterbi("北海道に住んでいる。")

(['', 'に', '住ん', 'で', 'いる', '。'], ['名詞', '名詞', '名詞', '名詞', '動詞', '名詞'])